In [2]:
import glob, json, pathlib
objs = glob.glob(r"D:\ig_pipeline\cad\objects\*\artifacts\objects\*\*\misc\metadata.json")
jsons = {}
for fn in objs:
    if pathlib.Path(fn).parts[-7].startswith("legacy_"):
        with open(fn, "r") as f:
            jsons[fn] = json.load(f)

In [3]:
len(jsons)

1245

In [4]:
ignore = set()

In [5]:
# Get the metadata keys
meta_keys_by_obj = {obj: set(metadict.keys()) for obj, metadict in jsons.items()}
all_meta_keys = {k for keys in meta_keys_by_obj.values() for k in keys}
print(all_meta_keys)

{'orientations', 'link_bounding_boxes', 'bbox_size', 'link_tags', 'meta_links', 'base_link_offset'}


In [6]:
[f for f, v in jsons.items() if "link_tags" not in v]

[]

In [7]:
len([f for f, v in meta_keys_by_obj.items() if "orientations" not in v])

0

In [8]:
meta_links_by_obj = {k: {m for link_keys in v["meta_links"].values() for m in link_keys.keys()} for k, v in jsons.items()}
all_meta_links = {m for links in meta_links_by_obj.values() for m in links}
print(all_meta_links)

{'particleremover', 'watersource', 'heatsource', 'cleaningtoolarea', 'slicer', 'togglebutton'}


In [38]:
# Get meta links from originals
import yaml, pathlib
TRANSLATION_PATH = r"C:\Users\Cem\research\iGibson-dev\igibson\data\ig_dataset\metadata\model_rename.yaml"
with open(TRANSLATION_PATH, "r") as f:
    TRANSLATION_DICT = yaml.load(f, Loader=yaml.SafeLoader)
    
orig_jsons = {}
for fn in objs:
    p = pathlib.Path(fn)
    key = p.parts[-4] + "/" + p.parts[-3]
    possible = [k for k, v in TRANSLATION_DICT.items() if v == key]
    if not possible:
        continue
    old_fn, = possible
    path = pathlib.Path(r"C:\Users\Cem\research\iGibson-dev\igibson\data\ig_dataset\objects") / old_fn / "misc/metadata.json"
    with open(path, "r") as f:
        orig_jsons[fn] = json.load(f)

In [41]:
orig_meta_keys_by_obj = {obj: set(metadict.keys()) - ignore for obj, metadict in orig_jsons.items()}
orig_all_meta_keys = {k for keys in orig_meta_keys_by_obj.values() for k in keys}
print(orig_all_meta_keys)

{'orientations', 'link_bounding_boxes', 'bbox_size', 'object_parts', 'openable_both_sides', 'base_link_offset', 'links', 'openable_joint_ids'}


In [82]:
[f for f, v in orig_meta_keys_by_obj.items() if "openable_both_sides" in v]

['D:\\ig_pipeline\\cad\\objects\\legacy_window-bhwwja\\artifacts\\objects\\window\\bhwwja\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-bxydsr\\artifacts\\objects\\window\\bxydsr\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-fufios\\artifacts\\objects\\window\\fufios\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-ithrgo\\artifacts\\objects\\window\\ithrgo\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-iwcqsz\\artifacts\\objects\\window\\iwcqsz\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-jzxtbv\\artifacts\\objects\\window\\jzxtbv\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-mjssrd\\artifacts\\objects\\window\\mjssrd\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-qdkcdn\\artifacts\\objects\\window\\qdkcdn\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_window-subxdn\\artifacts\\objects\\window\\subxdn\\misc\

In [9]:
orig_meta_links_by_obj = {k: (set(k.replace("_", "") for k in v["links"].keys()) if "links" in v else set()) for k, v in orig_jsons.items()}
orig_all_meta_links = {m for links in orig_meta_links_by_obj.values() for m in links}
print(orig_all_meta_links)

NameError: name 'orig_jsons' is not defined

In [88]:
[f for f, v in orig_meta_links_by_obj.items() if "watersource" in v and "waterdrain" not in v]

['D:\\ig_pipeline\\artifacts\\aggregate\\objects\\sink\\czyfhq\\misc\\metadata.json',
 'D:\\ig_pipeline\\artifacts\\aggregate\\objects\\sink\\ejooms\\misc\\metadata.json',
 'D:\\ig_pipeline\\artifacts\\aggregate\\objects\\sink\\ksecxq\\misc\\metadata.json',
 'D:\\ig_pipeline\\artifacts\\aggregate\\objects\\sink\\xiybkb\\misc\\metadata.json',
 'D:\\ig_pipeline\\artifacts\\aggregate\\objects\\sink\\zexzrc\\misc\\metadata.json']

In [87]:
[f for f, v in orig_jsons.items() if "links" in v and any(ml["geometry"] for ml in v["links"].values())]

['D:\\ig_pipeline\\cad\\objects\\legacy_broom-tpyvbt\\artifacts\\objects\\broom\\tpyvbt\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_carpet_sweeper-xboreo\\artifacts\\objects\\carpet_sweeper\\xboreo\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_carving_knife-alekva\\artifacts\\objects\\carving_knife\\alekva\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_carving_knife-awvoox\\artifacts\\objects\\carving_knife\\awvoox\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_carving_knife-mzhpkm\\artifacts\\objects\\carving_knife\\mzhpkm\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_pocketknife-ngtgdn\\artifacts\\objects\\pocketknife\\ngtgdn\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_scraper-icqmje\\artifacts\\objects\\scraper\\icqmje\\misc\\metadata.json',
 'D:\\ig_pipeline\\cad\\objects\\legacy_scrub_brush-hsejyi\\artifacts\\objects\\scrub_brush\\hsejyi\\misc\\metadata.json',
 'D:\\ig_pipeline\

In [ ]:
# Check each object
problematic = 0
for fn in objs:
    # Check meta links
    is_problematic = False
    if fn in orig_meta_links_by_obj:
        orig_meta_links = orig_meta_links_by_obj[fn]
        new_meta_links = meta_links_by_obj[fn]
        missing_meta_links = orig_meta_links - new_meta_links
        if missing_meta_links:
            print(fn, "missing meta links:", missing_meta_links)
            is_problematic = True
    if fn in orig_meta_keys_by_obj:
        orig_meta_keys = {(translation[x] if x in translation else x) for x in orig_meta_keys_by_obj[fn]}
        new_meta_keys = meta_keys_by_obj[fn]
        missing_meta_keys = orig_meta_keys - new_meta_keys
        if missing_meta_keys:
            print(fn, "missing meta keys:", missing_meta_keys)
            is_problematic = True
            
    if is_problematic:
        problematic += 1
        
print("Total problematic files:", problematic)
        

In [ ]:
orig_jsons[r"D:\ig_pipeline\artifacts\aggregate\objects\apple\agveuv\misc\metadata.json"]